In [1]:
from traceback import format_exc
from datetime import datetime
import json
import time
import os
import re

from easydict import EasyDict as edict
from bs4 import BeautifulSoup as bs
import requests as req
import pymysql as sql
import schedule
import redis

In [2]:
CONFIG_PATH = '/home/jovyan/project/utils/configs'
ROOT_PATH   = '/home/jovyan/project/DoveNest/informations'

load_json   = lambda path: edict(json.loads(open(path, 'r').read()))
CONFIGS     = load_json(f'{CONFIG_PATH}/config.json')
PORTS       = load_json(f'{CONFIG_PATH}/ports.json')

PASSWD      = CONFIGS.sql_passwd
HOST        = CONFIGS.global_host
USER        = CONFIGS.sql_user
PORT        = PORTS.sql_port

NIN_URL     = 'https://store.nintendo.co.kr/games/sale'
STE_URL     = 'https://store.steampowered.com/search/?specials=1&filter=topsellers&page='

NOW         = datetime.now()
Y, M, D     = NOW.year, NOW.month, NOW.day

RED_PORT    = PORTS.redis_port
red_conn    = redis.Redis(host = HOST, port = RED_PORT, 
                          decode_responses = True, db = 1)

In [3]:
class saleDB:
    
    def connect_db():
        
        conn   = sql.connect(host = HOST, port = PORT, user = USER, 
                            passwd=PASSWD, db = 'TEST')
        cursor = conn.cursor()
        
        return conn, cursor

In [4]:
conn, cursor = saleDB.connect_db()

In [5]:
cursor.execute('drop table discount_info')

0

In [6]:
query = f'''
            CREATE TABLE IF NOT EXISTS discount_info(
                idx   INTEGER NOT NULL,
                appid TEXT    NOT NULL,
                title TEXT    NOT NULL,
                storepage TEXT,
                thumbnail TEXT,
                percent INTEGER NOT NULL,
                original TEXT NOT NULL,
                discounted TEXT NOT NULL,
                platform TEXT NOT NULL,
                date TEXT NOT NULL);
        '''

cursor.execute(query)

0

In [7]:
res  = req.get(NIN_URL).text
soup = bs(res, 'html.parser') 

In [8]:
clear = lambda x: int(re.sub('[^0-9]*', '', x)) 
items = soup.select('div.category-product-item')

In [9]:
query      = '''
                INSERT INTO discount_info (idx, appid, title, storepage,
                thumbnail, percent, original, discounted, platform, date)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
             '''

for idx, item in enumerate(items):
    
    try:
        store_page = item.select('a.category-product-item-title-link')[0]['href']
        thumbnail  = item.select('span > img')[0]['data-src']
        original   = item.select('span > span.price')[1].text
        discount   = item.select('span > span.price')[0].text
        title      = item.select('a.category-product-item-title-link')[0].text.strip()
        appid      = store_page.split('.kr/')[1]
    
        platform   = 'nintendo'
        percent    = 100 - round(clear(discount) * 100 / clear(original))
        today      = f'{Y}{str(M).zfill(2)}{str(D).zfill(2)}'

        data_tuple = (idx, appid, title, store_page, thumbnail, percent,
                      original, discount, platform, today)
        
        cursor.execute(query, data_tuple)
    except Exception as e: print(e)

list index out of range
list index out of range
list index out of range


In [10]:
cursor.execute('select * from discount_info')

765

In [11]:
conn.commit()

In [12]:
whole_page = ''
for idx in range(1, 10):
    
    url         = f'{STE_URL}{idx}'
    whole_page += req.get(url).text
    
soup  = bs(whole_page, 'html.parser')
items = soup.select('div#search_resultsRows > a')

In [13]:
length = cursor.execute('select * from discount_info')
for idx, item in enumerate(items, 1):
    
    try:
        appid = item['data-ds-appid']
        title = item.select('span.title')[0].text
        
        store_page = f'https://store.steampowered.com/app/{appid}'
        json_      = json.loads(red_conn.get(f'id:{appid}'))
        thumbnail  = json_['data']['header_image']
        
        percent = item.select('.search_discount > span')[0].text
        percent = int(clear(percent))
        
        original = item.select('strike')[0].text.strip()
        discount = item.select('.discounted')[0].text.split('₩')[-1]
        discount = f'₩{discount}'
        
        platform = 'steam'
        today    = f'{Y}{str(M).zfill(2)}{str(D).zfill(2)}'
        


        
        data_tuple = (idx, appid, title, store_page, thumbnail, percent,
                      original, discount, platform, today)
    
        cursor.execute(query, data_tuple)
    except Exception as e: print(format_exc())

In [17]:
cursor.execute(f'select * from discount_info where date={today}')

990

In [15]:
conn.commit()